In [7]:
from tflearn import *
import tflearn
import tensorflow as tf
from PIL import Image
from collections import Counter
from glob import glob
import numpy as np
import cPickle
names = {0:'airplane', 1:'automobile', 2:'bird', 3:'cat', 4:'deer', 5:'dog', 6:'frog', 7:'horse', 8:'ship', 9:'truck'}

In [2]:
# Build the network and restore the trained model
imgAug = ImageAugmentation()
imgAug.add_random_flip_leftright()
imgAug.add_random_flip_updown()
imgAug.add_random_crop((28, 28))

imgPrep = ImagePreprocessing()
imgPrep.add_crop_center((28, 28))
imgPrep.add_zca_whitening

# Model constructor
def buildModel():
    network = input_data(shape=[None, 28, 28, 3], data_augmentation=imgAug, data_preprocessing=imgPrep)
    # CNN Block 1
    network = conv_2d(network, 32, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 32, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # CNN Block 2
    network = conv_2d(network, 64, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 64, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # CNN Block 3
    network = conv_2d(network, 128, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 128, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # CNN Block 4
    network = conv_2d(network, 160, 3)
    network = batch_normalization(network)
    network = relu(network)
    network = conv_2d(network, 160, 3)
    network = batch_normalization(network)
    network = relu(network)
    print(network.get_shape())
    network = tf.nn.fractional_max_pool(network, pooling_ratio=[1., 10/7., 10/7., 1.])[0]
    print(network.get_shape())
    
    # FC Block 1
    network = dropout(network, 0.5)
    network = fully_connected(network, 512, activation='relu')
    network = batch_normalization(network)
    network = fully_connected(network, 10, activation='softmax')
    network = regression(network, optimizer='Adam',
                         loss='categorical_crossentropy',
                         learning_rate=0.05)
    return network

net = buildModel()
model = tflearn.DNN(net, tensorboard_verbose=0)
model.load('./Final/model.tflearn')

(?, 28, 28, 32)
(?, 19, 19, 32)
(?, 19, 19, 64)
(?, 13, 13, 64)
(?, 13, 13, 128)
(?, 9, 9, 128)
(?, 9, 9, 160)
(?, 6, 6, 160)


In [3]:
# File loader from CIFAR10 website
def unpickle(file):
    fo = open(file, 'rb')
    dict = cPickle.load(fo)
    fo.close()
    return dict

In [4]:
# Load the test batch provided on the CIFAR10 site
data = unpickle('./Ver/test_batch')
xflat = data['data']
y = data['labels']

In [5]:
# Reshape the images into the 2d with 3 color channels and trim to size
x = []
for i in range(0,len(xflat)):
    im = np.reshape(xflat[i], (32,32,3), order='F')
    im = np.array(Image.fromarray(im).resize((28, 28)).rotate(270))
    x.append(im / 255.)

In [6]:
r = []
l = []
batch_size = 500

for iteration in range(0, 10):
    for b in range(0, 10000, batch_size):
        batch = x[b : (b + batch_size)]
        batch_labels = y[b : (b + batch_size)]
        r.extend(model.predict(batch))
        l.extend(batch_labels)

In [9]:
results = {'pred':r, 'act':l}
cPickle.dump(results, file('basecomp.pkl','w'))